In [10]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using LinearAlgebra, Plots
import ForwardDiff as FD
using Test
import Convex as cvx 
import ECOS
using Random
Random.seed!(1)

  Activating environment at `~/devel/recitations/2_17_recitation/Project.toml`


MersenneTwister(1)

# Convex.jl tutorial

This is convex modeling tool in Julia that let's us write out problems in a simple way, and then Convex.jl transforms them and sends them off to be solved (we're using [ECOS](https://github.com/embotech/ecos) as our solver today). If you want examples/inspiration for this technology, there are a few like this:

- Python: [CVXPY](https://www.cvxpy.org/) or [CVXOPT](http://cvxopt.org/) (cvxpy is probably what you want)
- Matlab: [CVX](http://cvxr.com/cvx/) or [YALMIP](https://yalmip.github.io/) (I like CVX better)
- R: [CVXR](https://cvxr.rbind.io/)

For Convex.jl the [repo is here](https://github.com/jump-dev/Convex.jl), and the [docs are here](https://jump.dev/Convex.jl/stable/)

These tools are just used for formulating your problem and verifying that it is Convex. The problem itself is solved by one of many available solvers, many common ones are:

- OSQP
- ECOS 
- CPLEX 
- Mosek 
- Gurobi
- COSMO 
- SeDuMi 
- SDPT3 
- GLPK 
- Hypatia 

## Least Squares 
For overdetermined systems (more equations than variables, "skinny" matrix A)
$$ \begin{align} \min_{x} \quad & \|Ax - b\|^2_2
 \end{align}$$

In [11]:
@testset "overdetermined" begin 
    # overdetermined
    A = randn(10,5)
    b = randn(10)
    x = cvx.Variable(5)
    
    prob = cvx.minimize(cvx.sumsquares(A*x - b)) # sumsquares(y) = dot(y,y) = norm(y)^2
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    xcvx = x.value::Matrix # This will always be a matrix
    xcvx = vec(x.value) # convert to vector easily 
    
    # compare with pseudoinverse
    @test norm(xcvx - (A'*A\(A'*b))) < 1e-4

end

Test Summary:  | Pass  Total
overdetermined |    1      1

ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -1.322e+00  +7e+00  5e-01  6e-02  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  -8.414e-02  -2.498e-01  +1e+00  9e-02  9e-03  2e-01  4e-01  0.7930  2e-02   1  1  1 |  0  0
 2  +6.991e-01  +1.012e+00  +1e+00  9e-01  4e-02  3e+00  3e-01  0.5543  6e-01   2  2  2 |  0  0
 3  +2.161e+00  +2.412e+00  +2e-01  1e-01  6e-03  6e-01  5e-02  0.8494  1e-02   2  1  1 |  0  0
 4  +6.859e-01  +1.938e+00  +1e-01  3e-01  9e-03  2e+00  4e-02  0.4696  5e-01   2  2  2 |  0  0
 5  +2.934e+00  +2.803e+00  +1e-01  6e-02  3e-03  4e-02  3e-02  0.5767  6e-01   2  2  2 |  0  0
 6  +4.186e+00  +4.179e+00  +1e-02  1e-02  5e-04  2e-02  3e-03  0.8948  4e-03   2  2  2 |  0  0
 7  +4.628e+00  +4.647e+00  +2e-03  4e-03  1e-04  3e-02  7e-04  0.8782  8e-02   2  2

Test.DefaultTestSet("overdetermined", Any[], 1, false, false)

For underdetermined systems (more variables than equations, "fat" matrix A)
$$ \begin{align} \min_{x} \quad & \|x\|^2_2 \\ 
 \text{st} \quad & A x = b 
 \end{align}$$

In [12]:
@testset "underdetermined" begin 
    
    # overdetermined
    A = randn(5,10)
    b = randn(5)
    x = cvx.Variable(10)
    prob = cvx.minimize(cvx.sumsquares(x))
    
    # add constraint 
    prob.constraints += (A*x == b)
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    xcvx = x.value::Matrix # This will always be a matrix
    xcvx = vec(x.value) # convert to vector easily 
    
    # compare with pseudoinverse
    @test norm(xcvx - A'*((A*A')\b)) < 1e-4


end

Test Summary:   | Pass  Total
underdetermined |    1      1

ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -1.322e+00  +7e+00  3e-01  7e-02  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  -6.948e-02  -2.844e-01  +2e+00  8e-02  1e-02  3e-01  5e-01  0.7428  2e-02   1  1  1 |  0  0
 2  +8.012e-01  +9.778e-01  +1e+00  5e-01  3e-02  2e+00  4e-01  0.7082  5e-01   2  2  2 |  0  0
 3  +1.660e+00  +2.051e+00  +1e-01  6e-02  3e-03  5e-01  4e-02  0.9860  9e-02   2  1  1 |  0  0
 4  +2.893e+00  +3.067e+00  +2e-02  1e-02  7e-04  2e-01  7e-03  0.8954  9e-02   2  2  2 |  0  0
 5  +3.168e+00  +3.279e+00  +1e-02  1e-02  4e-04  1e-01  3e-03  0.6701  3e-01   2  2  2 |  0  0
 6  +3.468e+00  +3.471e+00  +4e-04  3e-04  2e-05  4e-03  1e-04  0.9676  5e-03   3  1  2 |  0  0
 7  +3.481e+00  +3.481e+00  +3e-05  2e-05  1e-06  2e-04  9e-06  0.9890  6e-02   2 

Test.DefaultTestSet("underdetermined", Any[], 1, false, false)

## Equality constrained QP 

$$ \begin{align} \min_{x} \quad & \frac{1}{2} x^TQx + q^Tx \\ 
 \text{st} \quad & A x = b 
 \end{align}$$

In [13]:
let 
    
    n = 10 
    Q = randn(n,n); Q = Q'*Q + I # create PSD matrix 
    q = randn(n)
    
    A = randn(3,n)
    b = randn(3)
    
    x = cvx.Variable(n)
    
    # NOTE: quadform(x,Q) = x'*Q*x 
    cost = 0.5*cvx.quadform(x,Q) + dot(q,x) 
    
    prob = cvx.minimize(cost)
    
    prob.constraints += (A*x == b)
    
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    xcvx = x.value::Matrix # This will always be a matrix
    xcvx = vec(x.value) # convert to vector easily 
    
    
end
 


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +2.158e-01  -3.037e+01  +1e+02  4e-01  3e-01  1e+00  3e+01    ---    ---    1  3  - |  -  - 
 1  -3.434e-01  -1.277e+00  +6e+00  2e-02  1e-02  5e-01  2e+00  0.9552  3e-02   2  2  2 |  0  0
 2  -8.480e-01  -9.915e-01  +1e+00  4e-03  3e-03  3e-01  4e-01  0.8503  9e-02   2  2  2 |  0  0
 3  -7.309e-01  -7.334e-01  +2e-02  6e-05  6e-05  4e-03  7e-03  0.9890  6e-03   2  2  2 |  0  0
 4  -7.289e-01  -7.289e-01  +4e-04  1e-06  1e-06  7e-05  1e-04  0.9830  1e-04   2  1  2 |  0  0
 5  -7.289e-01  -7.289e-01  +1e-05  3e-08  3e-08  2e-06  4e-06  0.9679  1e-04   2  1  1 |  0  0
 6  -7.289e-01  -7.289e-01  +7e-07  2e-09  2e-09  1e-07  2e-07  0.9430  5e-04   3  1  1 |  0  0
 7  -7.289e-01  -7.289e-01  +7e-08  2e-10  2e-10  1e-08  2e-08  0.8932  9e-04   2  2  2 |  0  0
 8  -7.289e-01  -7.289e-01  +4e-09  2e-10  1e-

10-element Vector{Float64}:
  0.09724791987373017
  0.1488434193275145
  0.4860290018506496
  0.29931754044643294
 -0.106880331362541
  0.04208103746402248
 -0.1814010503222913
  0.3390224830173805
  0.12993335881168483
 -0.2421193804358406

## Letting Convex.jl do the parsing 

$$ \begin{align} \min_{x} \quad & \|Ax - b\|_1 \\ 
 \text{st} \quad &\|x\|_2 \leq 3
 \end{align}$$
 
 This problem is not in any sort of "standard form", but it is convex. We will let Convex.jl will convert this into a standard form "canonicalizing it", and send it ECOS to solve. 

In [14]:
let 
    A = randn(10,5)
    b = randn(10)
    x = cvx.Variable(5)
    
    prob = cvx.minimize(norm(A*x - b, 1)) 
    prob.constraints += (norm(x,2) <= 3)

    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    xcvx = x.value::Matrix # This will always be a matrix
    xcvx = vec(x.value) # convert to vector easily 
end


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +1.147e-17  -3.000e+00  +7e+01  5e-01  6e-01  1e+00  3e+00    ---    ---    1  1  - |  -  - 
 1  +4.051e+00  +4.241e+00  +2e+01  7e-02  1e-01  8e-01  8e-01  0.8261  1e-01   1  1  1 |  0  0
 2  +4.510e+00  +4.552e+00  +3e+00  1e-02  2e-02  2e-01  2e-01  0.8209  3e-02   1  1  1 |  0  0
 3  +4.534e+00  +4.533e+00  +1e-01  3e-04  5e-04  3e-03  5e-03  0.9890  2e-02   1  1  1 |  0  0
 4  +4.535e+00  +4.535e+00  +1e-03  4e-06  6e-06  3e-05  6e-05  0.9890  1e-04   1  1  1 |  0  0
 5  +4.535e+00  +4.535e+00  +1e-05  4e-08  6e-08  3e-07  7e-07  0.9890  1e-04   1  1  1 |  0  0
 6  +4.535e+00  +4.535e+00  +2e-07  4e-10  7e-10  4e-09  7e-09  0.9890  1e-04   1  0  0 |  0  0
 7  +4.535e+00  +4.535e+00  +2e-09  5e-12  8e-12  4e-11  8e-11  0.9890  1e-04   1  0  0 |  0  0

OPTIMAL (within feastol=7.8e-12, reltol=3.7e-

5-element Vector{Float64}:
  0.7522684827972297
  0.1491383275306395
  0.3234098585384891
  0.045071042424177546
 -0.4801159046796672

## Convex Trajectory Optimization
$$ \begin{align} \min_{x_{1:N},u_{1:N-1}} \quad & \sum_{i=1}^{N-1} \bigg[ \|x_i - x_g\|_2^2 + \|u_i\|_1 \bigg] + \frac{1}{2}x_N^TQ_fx_N & \\ 
 \text{st} \quad & x_1 = x_{\text{IC}} \\ 
 & x_{i+1} = A x_i + Bu_i \quad &\text{for } i = 1,2,\ldots,N-1 \\ 
 & x_N = x_g \\ 
 & \|u_i\|_2 \leq 3 \quad &\text{for } i = 1,2,\ldots,N-1\\ 
 & x_{min} \leq x_i \leq x_{max} \quad &\text{for } i = 1,2,\ldots,N-1\\ 
 \end{align}$$

In [15]:
function controllable(A,B)
    n = size(A,1)
    C = hcat([A^i*B for i = 0:(n-1)]...)
    return rank(C) == n 
end

let 
    
    # create linear system
    nx = 4 
    nu = 2 
    A = randn(nx,nx);
    B = randn(nx,nu);
    @assert controllable(A,B)
    
    # time steps 
    N = 20 
    x_ic = randn(nx)
    x_g = randn(nx)
    
    # terminal cost 
    Qf = randn(nx,nx); Qf = Qf'*Qf + I # make PSD Qf 
    
    # create cvx variables x_k = X[:,k], u_k = U[:,k]
    X = cvx.Variable(nx, N)
    U = cvx.Variable(nu, N - 1)
    
    # create cost 
    cost = 0 
    
    # stage cost 
    for k = 1:(N-1)
        xk = X[:,k]
        uk = U[:,k]
        cost += cvx.sumsquares(xk - x_g)
        cost += norm(uk, 1)
    end
    
    # terminal cost
    xn = X[:,N]
    cost += 0.5*cvx.quadform(xn, Qf)
    
    # initialize cvx problem 
    prob = cvx.minimize(cost)
    
    # initial condition constraint 
    prob.constraints += X[:,1] == x_ic 
    
    for k = 1:(N-1)
        # dynamics constraints 
        prob.constraints += (X[:,k+1] == A*X[:,k] + B*U[:,k])
    end
    
    # goal constraint 
    prob.constraints += X[:,N] == x_g
    
    # norm(u)<3 
    for k = 1:(N-1)
        uk = U[:,k]
        prob.constraints += norm(uk,2) <= 3 
    end
    
    x_min = -20*ones(nx)
    x_max =  20*ones(nx)
    for k = 1:N
        xk = X[:,k]
        prob.constraints += xk <= x_max 
        prob.constraints += xk >= x_min 
    end
    
    # solve problem (silent solver tells us the output)
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    if prob.status != cvx.MathOptInterface.OPTIMAL
        error("Convex.jl problem failed to solve for some reason")
    end
        
    # convert the solution matrices into vectors of vectors 
    X = X.value::Matrix
    U = U.value::Matrix
end


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -4.068e+03  +5e+03  6e-02  5e-01  1e+00  1e+01    ---    ---    1  2  - |  -  - 
 1  +3.000e+01  -1.290e+03  +2e+03  2e-02  2e-01  6e-01  5e+00  0.6746  4e-02   1  1  1 |  0  0
 2  +5.076e+01  -1.228e+03  +2e+03  2e-02  1e-01  1e+00  5e+00  0.1569  6e-01   2  2  2 |  0  0
 3  +7.483e+01  -1.108e+03  +1e+03  2e-02  9e-02  1e+00  4e+00  0.2373  6e-01   2  2  2 |  0  0
 4  +1.433e+02  -7.604e+02  +1e+03  1e-02  5e-02  3e+00  3e+00  0.7241  6e-01   2  1  1 |  0  0
 5  +1.226e+02  -5.024e+02  +7e+02  8e-03  3e-02  2e+00  2e+00  0.4020  3e-01   2  1  1 |  0  0
 6  +1.269e+02  -4.515e+02  +7e+02  8e-03  3e-02  2e+00  2e+00  0.2405  7e-01   2  2  2 |  0  0
 7  +1.121e+02  -2.328e+02  +4e+02  5e-03  1e-02  1e+00  1e+00  0.5832  3e-01   2  2  2 |  0  0
 8  +9.645e+01  +9.248e+00  +1e+02  1e-03  3e-

2×19 Matrix{Float64}:
 1.00046   -1.56966  -0.792683    …  -0.506034   -0.584798  -0.635175
 0.159675  -1.76543   2.11689e-9     -0.0434187  -0.236125  -1.2941